# Web Scraping
Isabela

In [13]:
import json
import tweepy
import pandas as pd
import time

import sys
import jsonpickle
import os

import praw


## Twitter

### Authentication

In [ ]:
with open('credentials/twitter_credentials') as f:
  credentials = json.load(f)
#print(credentials)

In [4]:
consumer_key = credentials["API_key"]
consumer_secret = credentials["API_secret_key"]
access_token = credentials["access_token"]
access_token_secret = credentials["access_token_secret"]
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)


In [5]:
text_query = 'COVID'
count = 150
try:
 # Creation of query method using parameters
 tweets = tweepy.Cursor(api.search,q=text_query + "-filter:retweets", lang="IT", locale="IT").items(count)

 # Pulling information from tweets iterable object
 tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]

 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
 tweets_df = pd.DataFrame(tweets_list)

except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

In [6]:
tweets_df

,0,1,2
0,2021-04-04 11:10:49,1378666361459597312,not saying there aren't any young people dying...
1,2021-04-02 15:04:47,1378000464147546112,L’Azienda Careggi sta contattando direttamente...
2,2021-04-01 04:19:42,1377475736672423936,i know y'all are enjoying pranking one another...
3,2021-03-30 10:28:01,1376843650873884674,Tre tipi di aziende su cui puntare nell'Europa...


Second Approach

In [12]:
searchQuery = 'covid'  # this is what we're searching for
maxTweets = 500000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'data/tweets1.txt' # We'll store the tweets in a text file.


# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None


places = []
time = []
tweets = []

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))

with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:

                print("No more tweets found")
                break
            for tweet in new_tweets:

                #add data to lists

                #1. creted at
                time.append((tweet.created_at.month,tweet.created_at.year))

                #location of user
                places.append(tweet.user.location)

                #text of tweet
                tweets.append(tweet.text)

                f.write(jsonpickle.encode(tweet.text, unpicklable=False) +'\n')

            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id

        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))


Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets
Downloaded 1100 tweets
Downloaded 1200 tweets
Downloaded 1300 tweets
Downloaded 1400 tweets
Downloaded 1500 tweets
Downloaded 1600 tweets
Downloaded 1700 tweets
Downloaded 1800 tweets
Downloaded 1900 tweets
Downloaded 2000 tweets
Downloaded 2100 tweets
Downloaded 2200 tweets
Downloaded 2300 tweets
Downloaded 2400 tweets
Downloaded 2500 tweets
Downloaded 2600 tweets
Downloaded 2700 tweets
Downloaded 2800 tweets
Downloaded 2900 tweets
Downloaded 3000 tweets
Downloaded 3100 tweets
Downloaded 3199 tweets
Downloaded 3299 tweets
Downloaded 3399 tweets
Downloaded 3499 tweets
Downloaded 3599 tweets
Downloaded 3699 tweets
Downloaded 3798 tweets
Downloaded 3898 tweets
Downloaded 3997 tweets
Downloaded 4097 tweets
Downloaded 4197 tweets
Downloaded 4297 tweets
Downloaded 4397 twee

## Reddit


### Authentication

In [18]:
with open('credentials/reddit_credentials') as f:
  credentials2 = json.load(f)

In [46]:
reddit = praw.Reddit(client_id=credentials2["client_id"], client_secret=credentials2["client_secret"], user_agent=credentials2['user_agent'])
fName = 'data/reddit10000.txt' # We'll store the tweets in a text file.



hot_posts = reddit.subreddit('Covid').top(limit=10000)
posts = 0
comments = 0
with open(fName, 'w') as f:
    for post in hot_posts:
        post.comments.replace_more(limit=None)
        #print("Title",post.title)
        f.write(jsonpickle.encode(post.title, unpicklable=False) +'\n')
        #print("Self Test",post.selftext)
        f.write(jsonpickle.encode(post.selftext, unpicklable=False) +'\n')
        posts = posts +1
        for top_level_comment in post.comments:
            #print("C:",top_level_comment.body)
            f.write(jsonpickle.encode(top_level_comment.body, unpicklable=False) +'\n')
            comments = comments + 1


print ("Downloaded {0} posts and {1} comments, Saved to {2}".format(posts,comments, fName))


Downloaded 1000 posts and 4059 comments, Saved to data/reddit10000.txt


In [45]:
reddit = praw.Reddit(client_id=credentials2["client_id"], client_secret=credentials2["client_secret"], user_agent=credentials2['user_agent'])
fName = 'data/reddit1.txt' # We'll store the tweets in a text file.



hot_posts = reddit.subreddit('Covid').top(limit=10000)
posts = 0
comments = 0
with open(fName, 'w') as f:
    for post in hot_posts:
        post.comments.replace_more(limit=None)
        #print("Title",post.title)
        f.write(jsonpickle.encode(post.title, unpicklable=False) +'\n')
        #print("Self Test",post.selftext)
        f.write(jsonpickle.encode(post.selftext, unpicklable=False) +'\n')
        posts = posts +1
        for top_level_comment in post.comments:
            #print("C:",top_level_comment.body)
            f.write(jsonpickle.encode(top_level_comment.body, unpicklable=False) +'\n')
            comments = comments + 1


print ("Downloaded {0} posts and {1} comments, Saved to {2}".format(posts,comments, fName))


Downloaded 1000 posts and 4059 comments, Saved to data/reddit1.txt
